In [1]:
import aidds.sys.config as cfg
from aidds.sys.utils.data_io import read_data, save_data
from aidds.sys.utils.evaluation import regression_evals
from aidds.sys.utils.exception import AiddsException
from sklearn.model_selection import train_test_split

In [2]:
ppdf = read_data(file_code='data.pp.last')

In [3]:
target_col = cfg.col.target
modeling_cols = ppdf.columns[2:].tolist()

In [4]:
check_df = ppdf[['acc_no', 'cons_cost']].copy()
X = ppdf[modeling_cols].copy()
y = ppdf[target_col].copy()
type(check_df), type(X), type(y)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series)

In [5]:
train_X, test_X, train_y, test_y = \
    train_test_split(X, y, test_size=0.25)

In [6]:
train_X.shape, test_X.shape

((10062, 98), (3354, 98))

In [7]:
yy = test_y.to_numpy()

In [8]:
# GBR
model = cfg.model.ml.gbr
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
_, message = regression_evals(y=yy, p=pred_y)
print(message)

MAPE: 0.188524, R2SCORE: 0.709301


In [9]:
pred_all_y = model.predict(X)
y_all = y.to_numpy()
_, message = regression_evals(y=y_all, p=pred_all_y)
print(message)

MAPE: 0.183535, R2SCORE: 0.738335


In [10]:
check_df['pred'] = pred_all_y

In [11]:
def calculate_mape(y, p):
    return abs((y-p)/y) * 100

check_df['mape'] = calculate_mape(check_df.cons_cost, check_df.pred)
check_df['best'] = check_df['mape'].apply(lambda x: '*' if x<=30 else '')

In [12]:
check_df.best.value_counts()

best
*    10963
      2453
Name: count, dtype: int64

In [13]:
best_df = check_df[check_df.best=='*'].copy()

In [14]:
best_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10963 entries, 0 to 13415
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   acc_no     10963 non-null  object 
 1   cons_cost  10963 non-null  int64  
 2   pred       10963 non-null  float64
 3   mape       10963 non-null  float64
 4   best       10963 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 513.9+ KB


In [15]:
ppdf.shape

(13416, 100)

In [16]:
ppdf = ppdf[ppdf.acc_no.isin(best_df.acc_no)]
ppdf.shape

(10963, 100)

In [17]:
save_data(ppdf, file_code='data.pp.best')